In [11]:
from rembg import remove
from PIL import Image,ImageDraw, ImageFont, ImageEnhance
import img2pdf
import PyPDF2
import numpy as np

imagem = Image.open(r'input.jpeg')
imagem.save(r'cropped.png')

# Caminho para a imagem de entrada
input_path = 'cropped.png'

# Caminho para a imagem de saída
output_path = 'nobg.png'

# Abre a imagem
input_image = Image.open(input_path)

# Ajusta o brilho e o contraste para saturar o branco
enhancer = ImageEnhance.Contrast(input_image)
enhanced_image = enhancer.enhance(100)  # Ajuste o fator conforme necessário

# Remove o fundo da imagem convertida
output_image = remove(input_image)

# Salva a imagem processada
output_image.save(output_path)

###############
# + REMOVENDO O FUNDO
##########

# Opening Image 
im = Image.open(r"cropped.png") 
  
# Creating object of Brightness class 
im3 = ImageEnhance.Brightness(im) 
  
# showing resultant image 
imgbr = im3.enhance(1.5)
enhancer = ImageEnhance.Contrast(imgbr)

#aumenta o contraste
enhanced_image = enhancer.enhance(2)

#deixa preto e branco
bw_image = enhanced_image.convert('L')

#covert to np array
input_array = np.array(bw_image)

# remove o fundo
output_array = remove(input_array)

# Create a PIL Image from the output array
output_image = Image.fromarray(output_array)
    
# Salva a imagem processada
output_image.save(output_path)

# + RECORTAR

image = output_image

# Converte a imagem para um array
pixels = image.getdata()

# Encontra os limites da área não transparente
left, top, right, bottom = image.width, image.height, 0, 0
for x in range(image.width):
    for y in range(image.height):
        # Verifica a transparência do pixel
        if pixels[y * image.width + x][3] > 0:
            left = min(left, x)
            top = min(top, y)
            right = max(right, x)
            bottom = max(bottom, y)

# Recorta a imagem na área não transparente
if left < right and top < bottom:
    cropped_image = image.crop((left, top, right + 1, bottom + 1))
    cropped_image

###############
# + REDIMENSIONANDO
###########

# Proporções originais
original_width, original_height = cropped_image.size

# Redimensiona para largura de 100 pixels, mantendo proporção
new_width = 100
new_height = int((new_width / original_width) * original_height)
nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)

# Verifica se a altura é maior que 40 pixels
if new_height > 50:
    # Redimensiona para altura de 40 pixels, mantendo proporção
    new_height = 50
    new_width = int((new_height / original_height) * original_width)
    nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)

###############
# + ADICIONA linha, sobrepõe imagens e adiciona textos
###########


# Carrega a imagem sem fundo
nobg_image = nobg_image_resized

# Dimensões da imagem nobg_image
nobg_width, nobg_height = nobg_image.size

# Carrega a imagem de fundo
background_image = Image.open("fundo.png")

# Cria um objeto Draw para desenhar na imagem de fundo
draw = ImageDraw.Draw(background_image)

# Coordenadas e dimensões da linha
line_x, line_y = 14, 50
line_width, line_height = 95, 1  # Largura de 95px e altura de 1px para a linha

# Desenha a linha
draw.line([(line_x, line_y), (line_x + line_width, line_y)], fill="black", width=line_height)

# Define as fontes para os textos
try:
    font_large = ImageFont.truetype("arial.ttf", 12)  # Fonte tamanho 8 para "Thiago Silva"
    font_small = ImageFont.truetype("arial.ttf", 8)  # Fonte tamanho 4 para "SST"
except IOError:
    font_large = ImageFont.load_default()
    font_small = ImageFont.load_default()

# Calcula a posição central da linha
center_line = line_x + line_width // 2

# Textos
text1 = "Thiago Silva"
text2 = "SST"

# Calcula a largura do texto para centralização
text1_width, text1_height = draw.textsize(text1, font=font_large)
text2_width, text2_height = draw.textsize(text2, font=font_small)

# Desenha o texto "Thiago Silva"
draw.text((center_line - text1_width // 2, line_y + 1), text1, font=font_large, fill="black")

# Desenha o texto "SST"
draw.text((center_line - text2_width // 2, line_y + text1_height + 1), text2, font=font_small, fill="black")

# Cola nobg_image centralizada sobre a linha, ajustada em 5 pixels para baixo
adjusted_y = line_y - nobg_height + 10
background_image.paste(nobg_image, (center_line - nobg_width // 2, adjusted_y), nobg_image)

# Salva
# Salva a imagem processada como PNG
background_image.save("final_image.png", "PNG")

# Nome do arquivo da imagem e do PDF
input_path = "final_image.png"
output_path = "final_image.pdf"

with open("final_image.png", "rb") as image_file:
    pdf = img2pdf.convert(image_file)

    with open("final_image.pdf", "wb") as output_file:
        output_file.write(pdf)

/tmp/ipykernel_148202/2124008971.py:94: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)
/tmp/ipykernel_148202/2124008971.py:101: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)
/tmp/ipykernel_148202/2124008971.py:143: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text1_width, text1_height = draw.textsize(text1, font=font_large)
/tmp/ipykernel_148202/2124008971.py:144: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text2_width, text2_height = draw.textsize(text2, font=font_small)
Image con

/tmp/ipykernel_148202/3619058642.py:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)
/tmp/ipykernel_148202/3619058642.py:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)


/tmp/ipykernel_148202/2628645315.py:39: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text1_width, text1_height = draw.textsize(text1, font=font_large)
/tmp/ipykernel_148202/2628645315.py:40: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text2_width, text2_height = draw.textsize(text2, font=font_small)
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
